In [407]:
import os
import json

def load_json_files(folder_path):
    data = []
    for folder_name in os.listdir(folder_path):
        folder_full_path = os.path.join(folder_path, folder_name)
        if os.path.isdir(folder_full_path):
            files = os.listdir(folder_full_path)
            for file_name in files:
                if file_name.endswith('.json'):
                    file_path = os.path.join(folder_full_path, file_name)
                    folder_split= folder_name.split("_")    

                    dataset = {
                        'model' : folder_split[0],
                        'threshold' : folder_split[-1],
                        'finetune' :  folder_split[2],
                        'dataset' : file_name.replace("_results.json" , ""),
                        'results' : None
                    }

                    with open(file_path, 'r') as f:
                        dataset['results'] = json.load(f)

                    data.append(dataset)
    return data

# Load all JSON files in the 'res' folder
folder_path = './results/'
data = load_json_files(folder_path)


In [408]:
import numpy as np
from itertools import groupby

def calculate_mean(data,key,mkey):
    m= []
    for item in data:
        if item['metrics'][key][mkey] is None or np.isnan(item['metrics'][key][mkey]):
            continue
        m.append(item['metrics'][key][mkey])
    if len(m) == 0:
        return None
    
    m = np.asarray(m)
    m = m.sum() / len(m)
    return float(round(m,2))

def calculate_variance(data, key, mkey):
    values = []
    for item in data:
        if item['metrics'][key][mkey] is None or np.isnan(item['metrics'][key][mkey]):
            continue
        values.append(item['metrics'][key][mkey])
    if len(values) == 0:
        return None
    
    values = np.asarray(values)
    variance = np.var(values)
    return float(round(variance, 2))

def calculate_median(data, key, mkey):
    values = []
    for item in data:
        if item['metrics'][key][mkey] is None or np.isnan(item['metrics'][key][mkey]):
            continue
        values.append(item['metrics'][key][mkey])
    if len(values) == 0:
        return None
    
    values = np.asarray(values)
    median = np.median(values)
    return float(round(median, 2))

def calculate_std_dev(data, key, mkey):
    values = []
    for item in data:
        if item['metrics'][key][mkey] is None or np.isnan(item['metrics'][key][mkey]):
            continue
        values.append(item['metrics'][key][mkey])
    if len(values) == 0:
        return None
    
    values = np.asarray(values)
    std_dev = np.std(values)
    return float(round(std_dev, 2))


def get_best(data,key,mkey, limit = 3):
    best = []
    for item in data:
        if item['metrics'][key][mkey] is None or np.isnan(item['metrics'][key][mkey]):
            continue

        best.append(
            {
                'filename': item['filename'],
                'path' : item['path'],
                'value': item['metrics'][key][mkey]
            }
        )
    if len(best) == 0:
        return None
    
    best = np.asarray(best)
    best = sorted(best, key=lambda x: x['value'])
    return best[-limit:], best[:-limit]


data_process = []
for d in data:
    # get list of all keys resulst
    sal_keys = list(d['results'][0]['metrics'].keys())
    for key in sal_keys:
        result= {
            'model' : d['model'],
            'dataset' : d['dataset'],
            'threshold' : d['threshold'],
            'finetune' : d['finetune'],
            'key' : key,
            'metrics' :  {},
            'statistics': {},
            'bests' : {},
            'worsts' : {}
        }   

        metrics_keys = list(d['results'][0]['metrics'][key].keys())
        for mkey in metrics_keys:
            if mkey== "statistics":
                continue
            mean__ = calculate_mean(d['results'],key,mkey)
            if mean__ is None:
                continue
            result['metrics'][mkey] = mean__
            bests, worts = get_best(d['results'],key,mkey)

            result['bests'][mkey] = bests
            result['worsts'][mkey] = worts

            result['statistics'][mkey] = {
                'variance' : calculate_variance(d['results'],key,mkey),
                'median' : calculate_median(d['results'],key,mkey),
                'std_dev' : calculate_std_dev(d['results'],key,mkey)
            }  

        data_process.append(result)


In [409]:
def groups_by_dataset(data):
    groups = {}
    for d in data:
        dataset = d['dataset']
        if dataset not in groups:
            groups[dataset] = []
        groups[dataset].append(d)
    return groups

def groups_by_model(data):
    groups = {}
    for d in data:
        model = d['model']
        if model not in groups:
            groups[model] = []
        groups[model].append(d)
    return groups

In [430]:
datasets_data = groups_by_dataset(data_process)

for dataset, dataset_values in datasets_data.items():
    # grousp by models
    models_data = groups_by_model(dataset_values)


    if "p3" not in dataset:
        continue

    print(f"DATASET: {dataset}")

    for model, model_values in models_data.items():
        model_values.sort(key=lambda x: x['threshold'])

        if model == "TranSalNetRes":
            continue

        print(f"{model}")


        for item in model_values:
            #  filter threshold
            if item['threshold'] != "0.5":
                continue

            # filter map
            # if item['key'] != "saliency" and item['key'] != "saliency_Add" and item['key'] != "rarity":
                continue

            print(f"Th: {item['threshold']} | Key: {item['key']} | {item['metrics']}")
            # print(f"     - statistics: {item['statistics']}")
        print("")


DATASET: p3_orientations
TranSalNetDense
Th: 0.5 | Key: saliency | {'msrt': 0.73, 'msrb': 1.66}
Th: 0.5 | Key: rarity | {'msrt': 0.94, 'msrb': 1.63}
Th: 0.5 | Key: saliency_Add | {'msrt': 0.87, 'msrb': 1.65}
Th: 0.5 | Key: saliency_Sub | {'msrt': 0.55, 'msrb': 2.27}
Th: 0.5 | Key: saliency_Prod | {'msrt': 0.93, 'msrb': 2.89}
Th: 0.5 | Key: saliency_Itti | {'msrt': 0.91, 'msrb': 1.75}

Unisal
Th: 0.5 | Key: saliency | {'msrt': 0.14, 'msrb': 48.75}
Th: 0.5 | Key: rarity | {'msrt': 2.21, 'msrb': 1.02}
Th: 0.5 | Key: saliency_Add | {'msrt': 0.82, 'msrb': 1.3}
Th: 0.5 | Key: saliency_Sub | {'msrt': 1.12, 'msrb': 1.12}
Th: 0.5 | Key: saliency_Prod | {'msrt': 0.41, 'msrb': 40.64}
Th: 0.5 | Key: saliency_Itti | {'msrt': 0.82, 'msrb': 1.33}

TempSal
Th: 0.5 | Key: saliency | {'msrt': 0.44, 'msrb': 131.77}
Th: 0.5 | Key: rarity | {'msrt': 0.75, 'msrb': 1.43}
Th: 0.5 | Key: saliency_Add | {'msrt': 0.79, 'msrb': 1.63}
Th: 0.5 | Key: saliency_Prod | {'msrt': 0.79, 'msrb': 26.49}
Th: 0.5 | Key: sali